In [1]:
import dotenv
import nest_asyncio

dotenv.load_dotenv()
nest_asyncio.apply()

In [ ]:
import os
from atproto import AsyncClient

client = AsyncClient()
await client.login(os.environ["BSKY_USR"], os.environ["BSKY_PWD"])

## List Trending Topics

In [33]:
topic_response = await client.app.bsky.unspecced.get_trending_topics()
topics = topic_response.topics

for topic in topics:
    print(topic.topic)
    print("-----")

NYC Mayoral Election
-----
ACA
-----
Jonathan Greenblatt
-----
UPS Crash
-----
Mamdani
-----
Quinnen Williams
-----
Max Dowman
-----
Dodgers
-----
World Series
-----
Blue Jays
-----


## Get posts from a specific feed

In [19]:
from atproto_client.models.app.bsky.feed.defs import FeedViewPost
from atproto_client.models.app.bsky.unspecced.defs import TrendingTopic
from atproto_client.models.app.bsky.feed.get_feed import Params as GetParams

async def _format_feed_link(topic: str | TrendingTopic) -> str:
    """Format the feed link for a given trending topic.
    Args:
        topic (TrendingTopic): The trending topic object.
    Returns:
        str: The formatted feed link.
    """
    topic = topic.link if isinstance(topic, TrendingTopic) else topic

    user_handle = topic.split('/')[4]
    feed_id = topic.split('/')[-1]
    user = await client.get_profile(user_handle)
    feed_link = f"at://{user.did}/app.bsky.feed.generator/{feed_id}"
    return feed_link

async def get_feed_posts(topic: str | TrendingTopic) -> list[FeedViewPost]:
    """Get posts from a specific feed based on a trending topic.
    Args:
        topic (TrendingTopic): The trending topic object.
    Returns:
        list[FeedViewPost]: A list of posts from the feed.
    """
    feed_link = await _format_feed_link(topic)
    feed = await client.app.bsky.feed.get_feed(GetParams(feed=feed_link, limit=100))
    return feed.feed

In [ ]:
feed = await get_feed_posts("https://bsky.app/profile/davehooper.com/feed/aaadhh6hwvaca") # Copied from trending topics on browser

In [29]:
for bsky_post in feed[:10]:
    print(bsky_post.post.record.text)
    print("-----")

GOP is the party of traumatizing toddlers. It's either kidnapping their teachers or sexually molesting them, but the GOP is all about it. 

What has the GOP ever done that hasn't traumatized toddlers? Because they really seem to enjoy starving them.
-----
Reporter: Democrats won in several high-profile elections yesterday, what do you think this means for the Trump administration and Republicans? 
Mike Johnson: There was an election yesterday? This is the first I'm hearing about it. I don't know anything about it.
-----
Today at the Supreme Court:

Supreme Court justices appear skeptical that Trump tariffs are legal

www.cnbc.com/2025/11/05/s...
-----
Is it so Trump knows where he is?
-----
Weil ich jetzt gerade Staffel 4 von "The Boys" schaue - ist irgendwie erschreckend realistisch geworden. Und Vought on Ice geht mir nicht mehr aus dem Kopf...
-----
Holy shit that NYC exit polling though, never let Dems tell you that queer people need to be abandoned to win, that’s a dirty lie.  We’

## Get all profiles a person is following

In [6]:
from tenacity import retry, wait_exponential, stop_after_attempt, retry_if_exception_type
from atproto_client.models.app.bsky.actor.defs import ProfileView

async def fetch_all_following(handle: str | ProfileView, client: AsyncClient, batch_limit=100, max_attempts=5) -> list[ProfileView]:
    """Fetch all followers for a handle using cursor-based pagination from the AT Protocol client.
    Args:
        client (AsyncClient): An authenticated instance of the AT Protocol client.
        batch_limit (int, optional): The maximum number of followers to fetch per request. Defaults to 100.
        max_attempts (int, optional): The maximum number of retry attempts for failed requests.
    Returns:
        list: A list of follower records.
    """
    if isinstance(handle, ProfileView):
        handle = handle.handle
        
    @retry(
        wait=wait_exponential(multiplier=1, min=4, max=10),
        stop=stop_after_attempt(max_attempts),
        retry=retry_if_exception_type(Exception),
    )
    async def fetch_batch(cursor=None):
        response = await client.get_follows(
            actor=handle,
            limit=batch_limit,
            cursor=cursor
        )
        return response
    
    all_following = []
    cursor = None
    
    while True:
        response = await fetch_batch(cursor)
        all_following.extend(response.follows)
        
        # Check if there are more pages
        if not response.cursor:
            break
        cursor = response.cursor

    return all_following

following = await fetch_all_following("mantzarlis.com", client)
print(f"Total following fetched: {len(following)}")

Total following fetched: 1219
